# Publications markdown generator for academicpages

Adapted from generator in academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

Note: edit in Excel and save as tsv. Then open in Notepad and save as with utf-8 encoding.

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,tags,download
0,2015-10-29,Comparing Global Link Arrangements for Dragonf...,2015 IEEE International Conference on Cluster ...,High-performance computing systems are shiftin...,"Hastings, E., Rincon-Cruz, D., Spehlmann, M., ...",dragonfly,https://ieeexplore.ieee.org/abstract/document/...,dragonfly,NaN
1,2018-11-05,Structure or Nurture? The Effects of Team-Buil...,Proceedings of the ACM on Human-Computer Inter...,How can instructors group students into teams ...,"Hastings, E. M., Jahanbakhsh, F., Karahalios, ...",psych_safety,https://dl.acm.org/citation.cfm?id=3274337,team,https://emhastings.github.io/files/structure-o...
2,2017-06-27,Learning Theory Exegesis: Collaborative Learning,CGScholar EPSY 408 SU17 Community,Teamwork has become increasingly popular in re...,"Hastings, Emily. (2017, July 17). Learning The...",collab_learning,https://cgscholar.com/community/profiles/user-...,other,NaN
3,2017-12-09,Introduction to Computer Science,Common Ground Research Networks Learning Desig...,This learning module is intended to be an intr...,"Hastings, Emily. (2017, December 9). Introduct...",intro-cs,https://cgscholar.com/bookstore/works/introduc...,other,NaN
4,2019-09-22,Agreement Behavior of Isolated Annotators for ...,Proceedings of the Annual Conference of the PH...,Maintenance work orders (MWOs) are an integral...,"Hastings, E., Sexton, T., Brundage, M. P., & H...",agreement,https://phmpapers.org/index.php/phmconf/articl...,nist,https://emhastings.github.io/files/agreement-b...
5,2020-04-25,LIFT: Integrating Stakeholder Voices into Algo...,Proceedings of the 2020 CHI Conference on Huma...,Team formation tools assume instructors should...,"Hastings, E. M., Alamri, A., Kuznetsov, A., Pi...",lift,https://dl.acm.org/doi/abs/10.1145/3313831.337...,team,https://emhastings.github.io/files/LIFT_CHI_20...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    if len(str(item.tags)) > 3:
        md += "\ntags: [" + html_escape(item.tags) + "]"
    
    md += "\n---\n"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Access online here](" + item.paper_url + ")\n" 
        
    if len(str(item.download)) > 5:
        md += "\n[Download author copy here](" + item.download + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)